## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
# already set up on Expanse; toggle for colab

# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

Set path prefix for Expanse

In [1]:
path_prefix = "/expanse/lustre/projects/nji102/sgriesmer/"

Import SNP datasets

In [2]:
import pandas as pd

abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/SNP_PRVCS_CAN_0_5023-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')
abs_diff_plus_log_odds_scores_norm.head(), abs_diff_plus_log_odds_scores_norm.shape


(   BroadDnd41CtcfUniPk151-ran  BroadDnd41Ezh239875UniPk151-ran  \
 0                    0.306653                        -0.556821   
 1                   -0.111710                        -0.113748   
 2                    0.090462                        -0.502083   
 3                   -0.143991                        -0.349744   
 4                   -0.144492                        -0.409669   
 
    BroadGm12878CtcfUniPk151-ran  BroadGm12878Ezh239875UniPk151-ran  \
 0                      0.018333                          -0.521790   
 1                      0.118644                           1.831607   
 2                     -0.059683                          -0.682626   
 3                     -0.189783                          -0.485974   
 4                     -0.191357                           0.003039   
 
    BroadH1hescChd1a301218aUniPk151-ran  BroadH1hescCtcfUniPk151-ran  \
 0                            -0.397740                     0.304906   
 1                      

Choose test set for run

In [3]:
import xgboost

In [4]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot

In [5]:
X=abs_diff_plus_log_odds_scores_norm[0:5023]

In [6]:
X

,BroadDnd41CtcfUniPk151-ran,BroadDnd41Ezh239875UniPk151-ran,BroadGm12878CtcfUniPk151-ran,BroadGm12878Ezh239875UniPk151-ran,BroadH1hescChd1a301218aUniPk151-ran,BroadH1hescCtcfUniPk151-ran,BroadH1hescEzh239875UniPk151-ran,BroadH1hescJarid1aab26049UniPk151-ran,BroadH1hescRbbp5a300109aUniPk151-ran,BroadHelas3CtcfUniPk151-ran,...,UwMcf7CtcfUniPk151-ran.1,UwNb4CtcfUniPk151-ran.1,UwNhdfneoCtcfUniPk151-ran.1,UwNhekCtcfUniPk151-ran.1,UwNhlfCtcfUniPk151-ran.1,UwRptecCtcfUniPk151-ran.1,UwSaecCtcfUniPk151-ran.1,UwSknshraCtcfUniPk151-ran.1,UwWerirb1CtcfUniPk151-ran.1,UwWi38CtcfUniPk151-ran.1
0,0.306653,-0.556821,0.018333,-0.521790,-0.397740,0.304906,-0.250733,-0.502371,-0.331445,0.104589,...,0.161162,2.493501,0.508397,0.315912,0.458727,0.510215,-0.314536,0.241683,0.519838,1.156624
1,-0.111710,-0.113748,0.118644,1.831607,-0.396543,-0.122070,-0.250106,-0.450897,-0.332165,4.180299,...,0.127964,1.306674,-0.299451,0.140796,1.378727,-0.102556,-0.500327,0.202275,0.561808,0.526119
2,0.090462,-0.502083,-0.059683,-0.682626,-0.397647,-0.163870,-0.250740,-0.502466,-0.332611,-0.136593,...,-0.257329,-0.208183,-0.278110,-0.138206,-0.198377,-0.124777,-0.306719,0.043669,-0.174972,-0.154073
3,-0.143991,-0.349744,-0.189783,-0.485974,-0.391880,-0.164242,-0.250665,-0.300571,-0.328237,-0.139470,...,1.188030,1.033966,-0.058070,-0.055008,0.305962,-0.300284,0.023444,-0.226416,-0.159670,3.054452
4,-0.144492,-0.409669,-0.191357,0.003039,-0.373176,-0.164270,-0.250392,-0.486401,-0.285498,-0.156234,...,-0.310854,-0.153070,-0.169165,-0.305359,-0.268203,-0.142192,-0.343744,-0.228562,-0.197016,-0.247948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,-0.138860,-0.543929,-0.073434,-0.687548,-0.397524,-0.155174,-0.250739,-0.495441,-0.332608,-0.130375,...,3.978750,0.303906,0.092633,0.585478,0.257663,0.429276,0.494833,4.729899,4.630994,-0.338688
5019,4.783827,1.997961,0.333336,1.850811,6.342943,2.034023,-0.247243,5.612971,-0.324075,3.024356,...,-0.300507,4.943333,2.558837,2.642982,1.639358,1.499459,1.607354,2.046898,1.752810,3.238786
5020,-0.140555,-0.553483,-0.152443,0.561306,0.647902,0.057787,-0.250562,0.076153,-0.331908,-0.037337,...,-0.199389,0.039086,-0.373643,0.314501,-0.158225,0.533938,-0.514059,-0.218508,0.153822,0.420841
5021,-0.143869,0.203208,-0.176924,1.938559,0.238020,-0.164218,-0.167718,-0.086010,0.060280,-0.152157,...,-0.274437,-0.208020,-0.395044,-0.216634,-0.241317,-0.117355,-0.343776,-0.205415,0.074402,-0.248003


In [7]:
col_lab = list(range(0,1380))
X.set_axis(col_lab, axis="columns", inplace=True)

In [8]:
X

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,0.306653,-0.556821,0.018333,-0.521790,-0.397740,0.304906,-0.250733,-0.502371,-0.331445,0.104589,...,0.161162,2.493501,0.508397,0.315912,0.458727,0.510215,-0.314536,0.241683,0.519838,1.156624
1,-0.111710,-0.113748,0.118644,1.831607,-0.396543,-0.122070,-0.250106,-0.450897,-0.332165,4.180299,...,0.127964,1.306674,-0.299451,0.140796,1.378727,-0.102556,-0.500327,0.202275,0.561808,0.526119
2,0.090462,-0.502083,-0.059683,-0.682626,-0.397647,-0.163870,-0.250740,-0.502466,-0.332611,-0.136593,...,-0.257329,-0.208183,-0.278110,-0.138206,-0.198377,-0.124777,-0.306719,0.043669,-0.174972,-0.154073
3,-0.143991,-0.349744,-0.189783,-0.485974,-0.391880,-0.164242,-0.250665,-0.300571,-0.328237,-0.139470,...,1.188030,1.033966,-0.058070,-0.055008,0.305962,-0.300284,0.023444,-0.226416,-0.159670,3.054452
4,-0.144492,-0.409669,-0.191357,0.003039,-0.373176,-0.164270,-0.250392,-0.486401,-0.285498,-0.156234,...,-0.310854,-0.153070,-0.169165,-0.305359,-0.268203,-0.142192,-0.343744,-0.228562,-0.197016,-0.247948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,-0.138860,-0.543929,-0.073434,-0.687548,-0.397524,-0.155174,-0.250739,-0.495441,-0.332608,-0.130375,...,3.978750,0.303906,0.092633,0.585478,0.257663,0.429276,0.494833,4.729899,4.630994,-0.338688
5019,4.783827,1.997961,0.333336,1.850811,6.342943,2.034023,-0.247243,5.612971,-0.324075,3.024356,...,-0.300507,4.943333,2.558837,2.642982,1.639358,1.499459,1.607354,2.046898,1.752810,3.238786
5020,-0.140555,-0.553483,-0.152443,0.561306,0.647902,0.057787,-0.250562,0.076153,-0.331908,-0.037337,...,-0.199389,0.039086,-0.373643,0.314501,-0.158225,0.533938,-0.514059,-0.218508,0.153822,0.420841
5021,-0.143869,0.203208,-0.176924,1.938559,0.238020,-0.164218,-0.167718,-0.086010,0.060280,-0.152157,...,-0.274437,-0.208020,-0.395044,-0.216634,-0.241317,-0.117355,-0.343776,-0.205415,0.074402,-0.248003


In [9]:
import pandas as pd

snp_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/CAN_dataset_XY-named.csv", sep=',')


In [10]:
y = snp_prvcs['label'][0:5023]

In [11]:
y

0       0
1       0
2       0
3       0
4       0
       ..
5018    1
5019    1
5020    1
5021    1
5022    1
Name: label, Length: 5023, dtype: int64

In [12]:
y[2500:2510]

2500    0
2501    0
2502    0
2503    0
2504    0
2505    0
2506    1
2507    1
2508    1
2509    1
Name: label, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [14]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

model1 = XGBClassifier() 
model2 = XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5) 
train_model1 = model1.fit(X_train, y_train) 
train_model2 = model2.fit(X_train, y_train) 
pred1 = train_model1.predict(X_test) 
pred2 = train_model2.predict(X_test) 
print("Accuracy_model1: %.4f" % (accuracy_score(y_test, pred1))) 
print("Accuracy_model2: %.4f" % (accuracy_score(y_test, pred2))) 
print("F1_model1: %.4f" % (f1_score(y_test, pred1)))
print("F1_model2: %.4f" % (f1_score(y_test, pred2)))
print("MCC_model1: %.4f" % (matthews_corrcoef(y_test, pred1)))
print("MCC_model2: %.4f" % (matthews_corrcoef(y_test, pred2)))
print("ROC_AUC_model1: %.4f" % (roc_auc_score(y_test, pred1))) # don't know about call: y_score vs. pred
print("ROC_AUC_model2: %.4f" % (roc_auc_score(y_test, pred2))) # don't know about call: y_score vs. pred


Accuracy_model1: 0.7094
Accuracy_model2: 0.7027
F1_model1: 0.7004
F1_model2: 0.6902
MCC_model1: 0.4245
MCC_model2: 0.4129
ROC_AUC_model1: 0.7114
ROC_AUC_model2: 0.7051


In [15]:
pred2.sum()

665

In [16]:
model3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=7)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy_model3: %.4f" % (accuracy_score(y_test, pred3)))
print("F1_model3: %.4f" % (f1_score(y_test, pred3)))
print("MCC_model3: %.4f" % (matthews_corrcoef(y_test, pred3)))
print("ROC_AUC_model3: %.4f" % (roc_auc_score(y_test, pred3))) # don't know about call: y_score vs. pred

Accuracy_model3: 0.7113
F1_model3: 0.7023
MCC_model3: 0.4286
ROC_AUC_model3: 0.7134


In [17]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[10],
 'min_child_weight':[1],
 'learning_rate':[0.1],
 'n_estimators':[1000],
 'objective':['binary:logistic']
 #'objective':['binary:logistic','binary:hinge','binary:logitraw']
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(gamma=0, 
                                               subsample=0.8, 
                                               colsample_bytree=0.8, 
                                               nthread=4, 
                                               scale_pos_weight=1, 
                                               seed=7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=5,
                       refit=True)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy_model4: %.4f" % (accuracy_score(y_test, pred4)))  
print("F1_model4: %.4f" % (f1_score(y_test, pred4)))  
print("MCC_model4: %.4f" % (matthews_corrcoef(y_test, pred4)))  
print("ROC_AUC_model4: %.4f" % (roc_auc_score(y_test, pred4)))  

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy_model4: 0.7153
F1_model4: 0.7043
MCC_model4: 0.4377
ROC_AUC_model4: 0.7176


In [18]:
pred4.sum()

670

In [19]:
gsearch.cv_results_['params'][gsearch.best_index_]

{'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 1000,
 'objective': 'binary:logistic'}

In [20]:
model_deepsea1 = XGBClassifier(
 reg_alpha = 20,
 reg_lambda = 2000,
 eta = 0.1,
 n_estimators=10,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea1 = model_deepsea1.fit(X_train, y_train)
pred_deepsea1 = train_model_deepsea1.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea1)))
print("F1_model_deepsea1: %.4f" % (f1_score(y_test, pred_deepsea1)))
print("MCC_model_deepsea1: %.4f" % (matthews_corrcoef(y_test, pred_deepsea1)))
print("ROC_AUC_model_deepsea1: %.4f" % (roc_auc_score(y_test, pred_deepsea1))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.6304
F1_model_deepsea1: 0.5400
MCC_model_deepsea1: 0.3063
ROC_AUC_model_deepsea1: 0.6384


In [21]:
model_deepsea2 = XGBClassifier(
 reg_alpha = 0,
 reg_lambda = 10,
 eta = 0.1,
 n_estimators=100,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea2 = model_deepsea2.fit(X_train, y_train)
pred_deepsea2 = train_model_deepsea2.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea2)))
print("F1_model_deepsea2: %.4f" % (f1_score(y_test, pred_deepsea2)))
print("MCC_model_deepsea2: %.4f" % (matthews_corrcoef(y_test, pred_deepsea2)))
print("ROC_AUC_model_deepsea2: %.4f" % (roc_auc_score(y_test, pred_deepsea2))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.7060
F1_model_deepsea2: 0.6980
MCC_model_deepsea2: 0.4173
ROC_AUC_model_deepsea2: 0.7080


In [22]:
# Save best model

In [23]:
from joblib import dump, load

best_estimator = train_model4.best_estimator_
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_CAN.joblib")

['/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_CAN.joblib']

In [24]:
test_estimator = load('/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_CAN.joblib')

In [25]:
test_pred = test_estimator.predict(X_test)

In [26]:
test_pred.sum()/1507

0.44459190444591906

In [27]:
# Load SNP data

In [27]:
SNPlt01_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/lt05_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [28]:
SNPlt01_abs_diff_plus_log_odds_scores_norm.shape

(10000, 1380)

In [29]:
SNPlt01_abs_diff_plus_log_odds_scores_norm = SNPlt01_abs_diff_plus_log_odds_scores_norm.iloc[:,0:1380:1]

In [30]:
SNPlt01_abs_diff_plus_log_odds_scores_norm.shape

(10000, 1380)

In [31]:
# Set model names to numbers

In [32]:
SNPlt01_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [33]:
SNPlt01_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [34]:
SNPlt01_pred = test_estimator.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [35]:
SNPlt01_pred[0:10000].sum()

8926

In [36]:
SNPgt5_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/SNPgt5_ran_10000_19997-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [37]:
SNPgt5_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [38]:
SNPgt5_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.138665,-0.420140,-0.173807,-0.640132,-0.354017,-0.167706,-0.201053,-0.433488,-0.293240,-0.152419,...,-0.298616,-0.195598,-0.173738,-0.237389,-0.234340,-0.230862,-0.462661,-0.233144,-0.201901,-0.270089
1,-0.138644,-0.336718,-0.173833,-0.386711,-0.357093,-0.167705,-0.201084,-0.408362,-0.291127,-0.152450,...,-0.233454,-0.195493,-0.381789,-0.232173,-0.229542,-0.293351,-0.216017,-0.233435,-0.189338,-0.293258
2,-0.138619,-0.500067,-0.173816,-0.511689,-0.355782,-0.167706,-0.199971,-0.367840,0.130106,-0.152452,...,-0.303600,-0.191994,-0.375076,-0.210616,-0.206243,-0.224043,-0.065949,-0.233740,-0.190216,-0.266444
3,-0.138608,-0.497402,-0.172060,0.577827,-0.337950,-0.166874,0.137476,-0.396060,-0.293746,-0.145622,...,-0.211894,-0.143020,-0.321114,-0.246540,0.221254,-0.208902,-0.173580,-0.234957,-0.200737,-0.177275
4,-0.138651,-0.476752,-0.173588,-0.557592,-0.356994,-0.167706,-0.201089,-0.427758,-0.286041,-0.152443,...,-0.079872,-0.125941,-0.195798,-0.134981,-0.216132,-0.201555,-0.451631,-0.230652,-0.184067,-0.165696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,-0.138425,-0.490933,-0.173668,-0.316845,-0.323496,-0.167704,-0.201039,-0.122288,-0.294265,-0.137659,...,-0.279311,-0.192734,-0.371469,-0.294450,-0.245204,-0.043556,0.851768,-0.233992,-0.189598,-0.273739
9993,-0.138602,2.429663,-0.173467,-0.634431,0.505936,-0.167705,-0.200044,-0.419610,-0.200065,-0.152306,...,-0.309292,-0.178383,-0.195712,-0.249831,-0.199768,-0.278973,-0.463986,-0.232627,-0.192912,-0.211255
9994,-0.137829,-0.501378,-0.173470,-0.477920,-0.293777,-0.167039,-0.201087,-0.416729,-0.262790,-0.150650,...,-0.051780,-0.195559,0.182378,0.146328,-0.019896,0.390730,-0.317636,-0.235096,-0.201179,-0.239532
9995,-0.138670,-0.493843,-0.173832,-0.638591,-0.357319,-0.167706,-0.201088,-0.437744,-0.294248,-0.152451,...,-0.314167,-0.201897,-0.379950,-0.293094,-0.253037,-0.281147,-0.460052,-0.230711,-0.196830,-0.298762


In [39]:
SNPgt5_pred = test_estimator.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [40]:
SNPgt5_pred.sum()/9997

0.854556366910073

In [41]:
# Try other models

In [42]:
pred1_lt01 = train_model1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred2_lt01 = train_model2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred3_lt01 = train_model3.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred4_lt01 = train_model4.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_lt01 = train_model_deepsea1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_lt01 = train_model_deepsea2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [47]:
for i in [
    pred1_lt01,
    pred2_lt01,
    pred3_lt01,
    pred4_lt01,
    pred_deepsea1_lt01,
    pred_deepsea2_lt01,
  ]:
    print(i.sum())
    

8987
8224
8677
8926
0
9507


In [45]:
pred1_gt5 = train_model1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred2_gt5 = train_model2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred3_gt5 = train_model3.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred4_gt5 = train_model4.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_gt5 = train_model_deepsea1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_gt5 = train_model_deepsea2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [46]:
for i in [
    pred1_gt5,
    pred2_gt5,
    pred3_gt5,
    pred4_gt5,
    pred_deepsea1_gt5,
    pred_deepsea2_gt5,
  ]:
    print(i.sum()/999)
    

9152
7317
8443
8543
0
9560
